In [ ]:
#@title Author: Michael Evans { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction

This notebook demonstrates a workflow for generating a map of predicted solar array footprints using a trained [fully convolutional neural network (FCNN)](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf), specifically [U-net](https://arxiv.org/abs/1505.04597) in Tensorflow. In this example, we create and export images that contain the same variables as used to train our model - the 3 visible, infrared, and 2 near-infrared bands of Sentinel-2 imagery from Google Earth Engine. We load the trained model structure and [weights](https://osf.io/eg35t/) and then run overlapping subsets of these images through the trained model to generate a 2-band output raster containing per-pixel probabilities and classes.

In [ ]:
import os
import shutil
import glob
from os.path import join
import ee

from tensorflow.python.keras import models
from sys import path
import numpy as np
import rasterio as rio
import json
from matplotlib import pyplot as plt
from matplotlib import colors
from tensorflow.python.keras import models

In [ ]:
# Authenticate and initiatlize GEE Account
ee.Authenticate()
ee.Initialize()

In [ ]:
## Clone repo containing preprocessing and prediction functions
!git clone https://github.com/mjevans26/Satellite_ComputerVision.git

In [ ]:
# Load the necessary modules from repo
path.append('./Satellite_ComputerVision')

In [ ]:
from utils.model_tools import get_model, make_confusion_matrix, weighted_bce
from utils.prediction_tools import doExport, makePredDataset, make_array_predictions, get_img_bounds, write_tfrecord_prediction, write_geotiff_prediction
from utils.clouds import basicQA

In [ ]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Specify names locations for outputs in Cloud Storage. 
BUCKET = '{YOUR_GCS BUCKET HERE}'
BUCKET_PATH = join('gs://', BUCKET)
FOLDER = '{YOUR PROJECT FOLDER HERE}'
PRED_BASE = '{YOUR PROJECT SUBDIRECTORY FOR PREDICTION FILES HERE}'
MODEL_PATH = '{PATH TO MODEL .h5 File}'
MODEL_WEIGHTS = '{PATH TO MODEL WEIGHTS .hdf5 file}'

# Specify inputs (Sentinel bands) to the model and the response variable.
opticalBands = ['B2', 'B3', 'B4']
thermalBands = ['B8', 'B11', 'B12']

BANDS = opticalBands + thermalBands

## Test images
We first need to create and export some images in GEE on which we can run predictions. This notebook uses a few test aois, but you can incorporate your own study areas in GEE or existing Sentinel-2 imagery

In [ ]:
# create several small aois to test predictions. These are all in NC
aois = dict({
    'Test1': ee.Geometry.Polygon(
        [[[-78.19610376358034, 35.086989862385884],
          [-78.19610376358034, 34.735631502732396],
          [-77.67974634170534, 34.735631502732396],
          [-77.67974634170534, 35.086989862385884]]], None, False),
    'Test2': ee.Geometry.Polygon(
        [[[-81.59087915420534, 35.84308746418702],
          [-81.59087915420534, 35.47711130797561],
          [-81.03057641983034, 35.47711130797561],
          [-81.03057641983034, 35.84308746418702]]], None, False),
    'Test3': ee.Geometry.Polygon(
        [[[-78.74447677513596, 36.4941960586897],
          [-78.74447677513596, 36.17115435938789],
          [-78.21713302513596, 36.17115435938789],
          [-78.21713302513596, 36.4941960586897]]], None, False),
    'Test4': ee.Geometry.Polygon(
        [[[-76.62411544701096, 36.33505523381603],
          [-76.62411544701096, 36.03800955668766],
          [-76.16818282982346, 36.03800955668766],
          [-76.16818282982346, 36.33505523381603]]], None, False)
})

In [ ]:
# Choose the GEE folder in which to ingest prediction image:
aoi = 'Test4'

# prediction path
test_path = join(FOLDER, PRED_BASE, aoi)

# Base file name to use for TFRecord files and assets. The name structure includes:
test_image_base = 'unet256_' + aoi

# Half this will extend on the sides of each patch.
kernel_buffer = [128, 128]

test_region = aois[aoi]

# find the center of our aoi for map visualization
center = test_region.centroid(5).coordinates().getInfo()
center.reverse()

In [ ]:
# Create a test image
S2 = ee.ImageCollection("COPERNICUS/S2")

## Change dates here
######
begin = '2020-05-01'
end = '2020-08-30'
######

# The image input collection is cloud-masked.
filtered = S2.filterDate(begin, end)\
.filterBounds(test_region)\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
.map(basicQA)

# Create a simple median composite to visualize
## Change .clip to change test area 
test = filtered.median().select(BANDS).clip(test_region)

# Use folium to visualize the imagery.
#mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
rgbParams = {'bands': ['B4', 'B3', 'B2'],
             'min': 0,
             'max': 3000}

nirParams = {'bands': ['B8', 'B11', 'B12'],
             'min': 0,
             'max': 3000}


## Change coordinates to center map based on aoi used 
map = folium.Map(location=center)
map.add_ee_layer(test, rgbParams, 'Color')
map.add_ee_layer(test, nirParams, 'Thermal')

map.add_child(folium.LayerControl())
map

In [ ]:
# Run the export.
## takes some time (~10 min) --> check GEE tasks to see when completed 
doExport(test, features = BANDS, pred_path = test_path, pred_base = test_image_base, scale = 10, bucket = BUCKET, region = test_region)

## Predictions

First we load the model structure and weights

In [ ]:
def get_weighted_bce(y_true,y_pred):
  return weighted_bce(y_true, y_pred, 1)
m = models.load_model(f'{MODEL_PATH}', custom_objects = {'get_weighted_bce': get_weighted_bce})
# m = get_model(depth = DEPTH, optim = OPTIMIZER, loss = get_weighted_bce, mets = METRICS, bias = None)
m.load_weights(f'{MODEL_WEIGHTS}')

Then generate a file list of our previously exported image data on which we want to make predictions. NOTE: This example reads from Google Cloud Storage, but any means of generating a list of filenames is sufficient

In [ ]:
predFiles = !gsutil ls {join(BUCKET_PATH, test_path, test_image_base + '*.tfrecord.gz')}
jsonFile = !gsutil ls {join(BUCKET_PATH, test_path, test_image_base + '*.json')}
jsonFile = jsonFile[0]

In [ ]:
# load our predictions data into a Dataset and inspect the first one
predData = makePredDataset(predFiles, BANDS, one_hot = None)
iterator = iter(predData)
print(iterator.next())

Generate and plot the output predictions

In [ ]:
# generate prediction rasters
preds = make_array_predictions(imageDataset = predData, model = m, jsonFile = jsonFile)

In [ ]:
# We can quickly visualize the predictions to see if they look sensible
figure = plt.figure(figsize = (12,12))

prob = preds[:, :, 0]
cls = out_image[:, :, 0]

plt.imshow(prob)

In [ ]:
# overlay the predicted outputs on the original satellite data map
heatmap = folium.raster_layers.ImageOverlay(
    image=prob,
    bounds= get_img_bounds(prob, jsonFile),
    colormap=lambda x: (0.5, 0, 0.5, 1) if x >= 0.9 else (0, 0, 0, 0),
)
map.add_child(heatmap)
map.add_child(folium.LayerControl())
map

Export and save predictions (optional)

In [ ]:
# optionally, write predictions to either tfrecord files (best for re-ingesting into GEE)...
write_tfrecord_predictions(predData, m, test_path, test_image_base)
#...or a geotiff
write_geotiff_predictions(image, jsonFile, '{OUTFILE}'):